In [1]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:1]

In [2]:
from sensitivity import test_sensitivity

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/symmetric')
asym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/asymmetric')

print('Symmetric results: ', sym_results)
print('Asymmetric results: ', asym_results)

INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:sensitivity:searching 9 coarse epsilons b/t 0.1 and 0.9
INFO:sensitivity:searching 9 coarse epsilons b/t 1.0 and 9.0
INFO:sensitivity:x0_s0 coarse epsilon bounds: (4.0, 5.0) (4367ms)
INFO:sensitivity:searching 100000 epsilons b/t 4.0 and 4.99999
INFO:sensitivity:x0_s0 epsilon: 4.1331 (1847ms)
INFO:sensitivity:x0 epsilon: (-4.1331, 4.1331) (6216ms)
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 and 

In [3]:
from tot_net import TOTNet

def check_input_epsilons(net, sample, x, le, ue, asym=False):
    inputs, outputs = sample
    y = outputs.index(max(outputs))
    lbs = inputs[0:x] + [inputs[x]+le] + inputs[x+1:]
    ubs = inputs[0:x] + [inputs[x]+ue] + inputs[x+1:]
    result = None
    if asym:
        result = (net.find_counterexample(lbs, inputs, y),
                  net.find_counterexample(inputs, ubs, y))
    else:
        result = net.find_counterexample(lbs, ubs, y)
    return result

sym_tests, asym_tests = {}, {}
net = TOTNet(nnet_path)
for x in range(net.get_num_inputs()):
    xid = f'x{x}'
    sym_tests[xid], asym_tests[xid] = [], []
    for sample in samples:
        sym_le, sym_ue = sym_results[xid][0]
        sym_cexs = check_input_epsilons(net, sample, x, sym_le, sym_ue)
        sym_tests[xid].append(sym_cexs)

        asym_le, asym_ue = asym_results[xid][0]
        asym_cexs = check_input_epsilons(net, sample, x, asym_le, asym_ue, asym=True)
        asym_tests[xid].append(asym_cexs)

print('Symmetric tests:')
for x in sym_tests:
    print(x, sym_tests[x] if any(sym_tests[x]) else 'ok')

print('Asymmetric tests:')
for x in asym_tests:
    print(x, asym_tests[x] if any([l or u for l,u in asym_tests[x]]) else 'ok')

Symmetric tests:
x0 ok
x1 ok
x2 ok
x3 ok
x4 ok
x5 ok
x6 ok
x7 ok
x8 ok
x9 ok
x10 ok
x11 ok
x12 ok
x13 ok
x14 ok
x15 ok
x16 ok
x17 ok
x18 ok
x19 ok
x20 ok
x21 ok
x22 ok
x23 ok
x24 ok
Asymmetric tests:
x0 ok
x1 ok
x2 ok
x3 ok
x4 ok
x5 ok
x6 ok
x7 ok
x8 ok
x9 ok
x10 ok
x11 ok
x12 ok
x13 ok
x14 ok
x15 ok
x16 ok
x17 ok
x18 ok
x19 ok
x20 ok
x21 ok
x22 ok
x23 ok
x24 ok
